In [5]:
from linkedin import Scraper

# selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# other
import pandas as pd
import time

# credentials
email = 'joseph.barbierdarnal@gmail.com'
with open('../credentials.txt', 'r') as file:
    password = file.read()

In [7]:
scraper = Scraper()
scraper.load_page("https://www.linkedin.com/")
scraper.accept_cookies()
scraper.login(email, password)

MaxRetryError: HTTPConnectionPool(host='localhost', port=56745): Max retries exceeded with url: /session/E30EC8E1-0BDC-4BA0-94B8-033A16C4449B/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x13b2b3700>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [6]:
scraper.load_page(
    "https://www.linkedin.com/jobs/collections/recommended/",
    try_quitting_first=False
)
scraper.close_message()

scraper.enter_keywords('data science')
scraper.enter_location('European Economic Area')
scraper.press_enter_and_scroll()
scraper.scrap_jobs()

In [5]:
pd.read_csv('../data/job_df.csv')

,Job ID,Title,Company,Location,Description,Date
0,3782287068,"Business Intelligence Engineer II, S&OP Automa...",Amazon,"Luxembourg, Luxembourg, Luxembourg","Business Intelligence Engineer II, S&OP Automa...",
1,3775375662,Internship Artificial Intelligence & Computer ...,Korro AI,"Darmstadt, Hesse, Germany (Hybrid)",Internship Artificial Intelligence & Computer ...,
2,3782579723,Consultant Data Science (all genders),Accenture DACH,"Kronberg, Hesse, Germany (On-site)",Consultant Data Science (all genders)\nAccentu...,
3,3781591727,Data Scientist for A320 Efficiency Leader (M/F),Airbus,"Toulouse, Occitanie, France (Hybrid)",Data Scientist for A320 Efficiency Leader (M/F...,
4,3790751316,Data Engineer,'s Heeren Loo,"Amersfoort, Utrecht, Netherlands (On-site)","Data Engineer\n's Heeren Loo\nAmersfoort, Utre...",
...,...,...,...,...,...,...
279,3784864205,Senior Machine Learning Engineer,Ani Biome,"Zagreb, Zagreb, Croatia (On-site)",Senior Machine Learning Engineer\nAni Biome\nZ...,2 weeks ago
280,3784559034,Lecturer in Data/Machine Learning Engineering ...,Breda University of Applied Sciences,"Breda, North Brabant, Netherlands (On-site)",Lecturer in Data/Machine Learning Engineering ...,2 weeks ago
281,3651025688,Product Data Scientist Intern,Criteo,"Paris, Île-de-France, France (Hybrid)","Product Data Scientist Intern\nCriteo\nParis, ...",2 weeks ago
282,3775702364,Senior Analyst,INVL Asset Management,"Vilnius, Vilniaus, Lithuania (Hybrid)",Senior Analyst\nINVL Asset Management\nVilnius...,2 weeks ago
